In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
#import fiona 
#import geopandas as gpd
#import math
#from netCDF4 import Dataset
#import xarray as xa
import matplotlib.pyplot as plt
%matplotlib inline
#from matplotlib import cm
#from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [ ]:
tmax_path = 'c:/Users/markstro/work1.1/cbh_gridmet/tmax.cbh'
prcp_path = 'c:/Users/markstro/work1.1/cbh_gridmet/prcp.cbh'
gdb_path = 'c:/Users/markstro/work1.1/GIS/GFv1.1_v2e.gdb'

#fns = [("2011", "2011_gm_climate_2020_03_02.nc"), ("2012", "2012_gm_climate_2020_03_03.nc"),
#       ("2013", "2013_gm_climate_2020_03_05.nc"), ("2014", "2014_gm_climate_2020_03_06.nc"),
#       ("2015", "2015_gm_climate_2020_03_08.nc"), ("2016", "2016_gm_climate_2020_03_09.nc"),
#       ("2017", "2017_gm_climate_2020_03_10.nc"), ("2018", "2018_gm_climate_2020_03_12.nc")]

param_path = "c:/Users/markstro/work1.1/paramdb_v1.1/paramdb_master/"

In [ ]:
# Read the cbh file
def read_cbh(cbh_fn):
#    foo = np.loadtxt(cbh_fn, delimiter=' ', skiprows=3,)
    foo = np.loadtxt(cbh_fn, skiprows=3,)

    dates = foo[:,0:6].astype('int')
    vals = foo[:,6:foo.shape[1]]
    
    return dates, vals

In [ ]:
# Temperature is in C
dates, tmax = read_cbh(tmax_path)

In [ ]:
print(tmax.shape)
tmax_f = tmax * 1.8 +32
print(tmax_f.shape)

In [ ]:
print(dates)
print(tmax)
print(tmax.shape)
print(min(tmax.flatten()),max(tmax.flatten()))
print(min(tmax_f.flatten()),max(tmax_f.flatten()))

In [ ]:
# prcp is in mm/day
dates, prcp = read_cbh(prcp_path)

In [ ]:
print(dates)
print(prcp)
print(prcp.shape)

ndays = prcp.shape[0]
nhru = prcp.shape[1]

print("ndays = ", ndays)
print("nhru = ", nhru)

In [ ]:
prcp_in = prcp / 25.4

In [ ]:
foo = prcp_in.flatten()
print(min(foo), max(foo))


In [ ]:
foo = foo[foo > 0.01]
foo = foo[foo < 5]
print(min(foo), max(foo))
plt.hist(foo, )
plt.show()

In [ ]:
#nhru_v11 = gpd.read_file(gdb_path, layer='nhru_v11')

In [ ]:
#nhru_v11_ids = nhru_v11["nhru_v11"].values
#nhru = len(nhru_v11_ids)
#print(nhru)
#print(nhru_v11_ids)

#nmonths = 12
#print(nhru * nmonths)

In [ ]:
nmonths = 12

In [ ]:
# Read PRMS parameters
tmax_allrain_offset_f = pd.read_csv(param_path + "tmax_allrain_offset.csv")
tmax_allrain_offset = tmax_allrain_offset_f["tmax_allrain_offset"].values

tmax_allsnow_f = pd.read_csv(param_path + "tmax_allsnow.csv")
tmax_allsnow = tmax_allsnow_f["tmax_allsnow"].values
tmax_allrain = tmax_allsnow + tmax_allrain_offset

print(tmax_allrain)
print(tmax_allrain.shape)

tmax_allrain.shape = (nmonths, nhru)

print(tmax_allrain.shape)

# These 2D parameters are all set to constants
ppt_rad_adj = 0.02
tmax_index = 50.0
radj_sppt = 0.44
radj_wppt = 0.55
radadj_intcp = 1.0
radadj_slope = 0.02

In [ ]:
#nhru_v11.head()

In [ ]:
#! Set precipitation adjument factor based on temperature
#! and amount of precipitation
# pptadj = 1.0
# IF ( Hru_ppt(j)>Ppt_rad_adj(j,Nowmonth) ) THEN
# IF ( Tmax_hru(j)<Tmax_index(j,Nowmonth) ) THEN
#   pptadj = Radj_sppt(j)
#   IF ( Tmax_hru(j)>=Tmax_allrain(j,Nowmonth) ) THEN
#     IF ( Summer_flag==0 ) pptadj = Radj_wppt(j) ! Winter
#   ELSE
#     pptadj = Radj_wppt(j)
#   ENDIF
# ELSE
#   pptadj = Radadj_intcp(j, Nowmonth) + &
# &                 Radadj_slope(j, Nowmonth)*(Tmax_hru(j)-Tmax_index(j,Nowmonth))
#   IF ( pptadj>1.0 ) pptadj = 1.0
# ENDIF
# ENDIF

pptadj = np.zeros(ndays*nhru)
pptadj.shape = (ndays,nhru)

for iday in range(ndays):
    yr = dates[iday,0]
    mo = dates[iday,1]
    day = dates[iday,2]
    date = datetime(yr,mo,day)
    jday = date.timetuple().tm_yday
    imon = date.month - 1
    summer_flag = True
    if jday < 79 or jday > 265:
        summer_flag = False
#    print(iday, yr, mo, day, date, jday, imon, summer_flag)
    
    for ihru in range(nhru):
        pptadj[iday,ihru] = 1.0
#        print(prcp_in[iday,ihru], ppt_rad_adj)
        if prcp_in[iday,ihru] > ppt_rad_adj:
#            print(tmax[iday,ihru], tmax_index)
            if tmax_f[iday,ihru] < tmax_index:
                pptadj[iday,ihru] = radj_sppt
                if tmax_f[iday,ihru] >= tmax_allrain[imon,jday]:
                    if not summer_flag:
                        pptadj[iday,ihru] = radj_wppt
                else:
                    pptadj[iday,ihru] = radj_wppt
            else:
                pptadj[iday,ihru] = radadj_intcp + radadj_slope * (tmax_f[iday,ihru] - tmax_index)
#                print(pptadj[iday,ihru])
                if pptadj[iday,ihru] > 1.0:
                    pptadj[iday,ihru] = 1.0

In [ ]:
foo = pptadj.flatten()

In [ ]:
print(foo.shape)
print(min(foo), max(foo))